In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [3]:
# xData => [공부시간, 과외시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # 7행 2열
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [4]:
# yData => [합격여부] => 실제값
temp = [0, 0, 0, 1, 1, 1, 1] # 1행 7열
print(temp)
print(type(temp))

yData = np.array(temp)
print(yData)
print(type(yData))

# reshape(): numpy에서 데이터는 그대로 유지한채 배열의 형태(차원)을 변경한다.
# 1행 7열 numpy 배열 => 7행 1열 numpy 배열
yData = np.array(temp).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [5]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성으로 대입되는 배열의 차원을 지정해야 한다.
# [None, 2] => 행의 개수는 상관없고 열의 개수는 2개.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData

가중치와 a와 y절편 b를 임의의 값으로 정한다.



In [6]:
a = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32) # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a1: {:f}, a2: {:f}, b: {:f}'.format(sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')

a1: 0.006021, a2: 0.382725, b: 0.676602
a
[[0.00602067]
 [0.38272452]]
b
[0.67660236]


시그모이드 방정식, 오차 함수, 경사 하강법

In [7]:
# sigmoid(): 시그모이드 방정식.
# matmul(): 행렬의 곱(적)을 계산한다. matrix multiply 
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식. y=예측값.
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강법

sigmoid() 함수의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 -1을 리턴시킨다.

In [9]:
sess = tf.Session()
# cast(x, dtype): x에 저장된 데이터를 dtype으로 형변환한다.
# predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
# print(sess.run(predict)) # [1 2]
# cast() 함수는 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 형변환한다.
# predict = tf.cast([True, False], dtype=tf.int32)
# print(sess.run(predict)) # [1 0]
predict = tf.cast([0.4 >= 0.5, 0.5 >= 0.5], dtype=tf.int32)
print(sess.run(predict))

[0 1]


sigmoid() 함수를 실행한 예상값(0 또는 1)을 계산한다.



In [12]:
predict = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, T/F를 int로 형변환.
print(predict)

Tensor("Cast_3:0", shape=(?, 1), dtype=float32)


예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [14]:
sess = tf.Session()
# equal(): 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0))) # True False
# 정확도 : 예측값과 실제값 비교 후 T/F를 1/0으로 캐스팅하고 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.int32)) # 예측값 vs 실제값


True False


In [16]:
# 학습시킨다.

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(3001):
    # sess.run([실행할내용, ...], feed_dict={플레이스홀더 이름: 입력 데이터})
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if epoch % 300 == 0:
        print('epoch: {:5d}, a1: {:9.5f}, a2: {:9.5f}, b: {:9.5f}, loss: {:9.5f}'.format(epoch, a_[0][0], 
            a_[1][0], b_[0], loss_))

epoch:     0, a1:   0.73069, a2:   0.05220, b:   0.41530, loss:   2.02282
epoch:   300, a1:   0.89405, a2:  -0.68295, b:  -2.33077, loss:   0.27338
epoch:   600, a1:   0.86912, a2:  -0.37482, b:  -3.80911, loss:   0.19621
epoch:   900, a1:   0.76776, a2:  -0.02915, b:  -4.88495, loss:   0.15302
epoch:  1200, a1:   0.65684, a2:   0.28933, b:  -5.73656, loss:   0.12494
epoch:  1500, a1:   0.55276, a2:   0.57124, b:  -6.44247, loss:   0.10526
epoch:  1800, a1:   0.45947, a2:   0.81877, b:  -7.04562, loss:   0.09076
epoch:  2100, a1:   0.37707, a2:   1.03654, b:  -7.57227, loss:   0.07969
epoch:  2400, a1:   0.30453, a2:   1.22925, b:  -8.03974, loss:   0.07098
epoch:  2700, a1:   0.24051, a2:   1.40098, b:  -8.46007, loss:   0.06396
epoch:  3000, a1:   0.18374, a2:   1.55517, b:  -8.84191, loss:   0.05819
